<a href="https://colab.research.google.com/github/rsamala/Bots/blob/main/Wprking%20Chatbot_with_PDF_Youtube.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Introduction

This notebook has all the code you need to create your own chatbot with custom knowledge base using GPT-3. 

Follow the instructions for each steps and then run the code sample. In order to run the code, you need to press "play" button near each code sample.

#Download the data for your custom knowledge base
For the demonstration purposes we are going to use ----- as our knowledge base. You can download them to your local folder from the github repository by running the code below.
Alternatively, you can put your own custom data into the local folder. 

# Install the dependicies
Run the code below to install the depencies we need for our functions

In [7]:
!pip install llama-index


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.0/160.0 KB 11.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 450.7/450.7 KB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 KB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 74.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 70.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 KB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 105.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.5/135.5 KB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 K

In [8]:
!pip install PyPDF2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 KB 11.2 MB/s eta 0:00:00


In [9]:
!pip install langchain

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
# youtube requirements
!pip install youtube_transcript_api
import sys, re
from youtube_transcript_api import YouTubeTranscriptApi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
#define youtube related functions
def get_video_transcript(video_id):
    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id)
        return transcript
    except Exception as e:
        print("Error:", e)
        return None

def save_transcript_to_file(transcript, output_file):
    with open(output_file, "w") as f:
        for entry in transcript:
            caption_text = entry["text"]
            # Remove special characters and line breaks, then append a space.
            cleaned_text = re.sub(r'[\n\]]', '', caption_text).replace('\n', ' ') + ' '
            f.write(cleaned_text)

def extract_video_id(url):
    patterns = [
        r"(?:https?:\/\/)?(?:www\.)?youtube\.com\/watch\?v=([^&]+)",
        r"(?:https?:\/\/)?(?:www\.)?youtu\.be\/([^&]+)",
        r"(?:https?:\/\/)?(?:www\.)?youtube\.com\/embed\/([^&]+)",
        r"(?:https?:\/\/)?(?:www\.)?youtube\.com\/v\/([^&]+)",
    ]

    for pattern in patterns:
        match = re.search(pattern, url)
        if match:
            return match.group(1)

    return None

In [5]:

#Get Content into required folder /content/sample_data/Data"
url=input("Paster youtube url")
video_id = extract_video_id(url)

if url:
  transcript = get_video_transcript(video_id)

output_file = f"{video_id}_transcript.txt"
FQFN = "/content/sample_data/Data/" +  output_file

if transcript:
  save_transcript_to_file(transcript, FQFN)
  print(f"Transcript saved to {output_file}")
else:
  print("Failed to fetch transcript")

Paster youtube urlhttps://youtu.be/_v_fgW2SkkQ
Transcript saved to _v_fgW2SkkQ_transcript.txt


# Define the functions
The following code defines the functions we need to construct the index and query it

In [10]:
from llama_index import SimpleDirectoryReader, GPTListIndex, readers, GPTSimpleVectorIndex, LLMPredictor, PromptHelper
from langchain import OpenAI
import sys
import os
from IPython.display import Markdown, display
from llama_index import ServiceContext

def construct_index(directory_path):
    # set maximum input size
    max_input_size = 4096
    # set number of output tokens
    num_outputs = 2000
    # set maximum chunk overlap
    max_chunk_overlap = 20
    # set chunk size limit
    chunk_size_limit = 600 

    # define LLM
    llm_predictor = LLMPredictor(llm=OpenAI(temperature=0.5, model_name="text-davinci-003", max_tokens=num_outputs))
    prompt_helper = PromptHelper(max_input_size, num_outputs, max_chunk_overlap, chunk_size_limit=chunk_size_limit)
 
    documents = SimpleDirectoryReader(directory_path).load_data()
    
#    index = GPTSimpleVectorIndex(
#        documents, llm_predictor=llm_predictor, prompt_helper=prompt_helper
#    )
    service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, prompt_helper=prompt_helper)
    index = GPTSimpleVectorIndex.from_documents(documents, service_context=service_context)

    index.save_to_disk('index.json')

    return index

def ask_ai():
    index = GPTSimpleVectorIndex.load_from_disk('index.json')
    while True: 
        query = input("What do you want to ask? ")
        response = index.query(query, response_mode="compact")
        display(Markdown(f"Response: <b>{response.response}</b>"))
  

# Set OpenAI API Key
You need an OPENAI API key to be able to run this code.

If you don't have one yet, get it by [signing up](https://platform.openai.com/overview). Then click your account icon on the top right of the screen and select "View API Keys". Create an API key.

Then run the code below and paste your API key into the text input.

In [14]:
os.environ["OPENAI_API_KEY"] = input("Paste your OpenAI key here and hit enter:")

KeyboardInterrupt: ignored

#Construct an index
Now we are ready to construct the index. This will take every file in the folder 'data', split it into chunks, and embed it with OpenAI's embeddings API.

**Notice:** running this code will cost you credits on your OpenAPI account ($0.02 for every 1,000 tokens). If you've just set up your account, the free credits that you have should be more than enough for this experiment.

In [12]:
construct_index("/content/sample_data/Data")

#Ask questions
It's time to have fun and test our AI. Run the function that queries GPT and type your question into the input. 

If you've used the provided example data for your custom knowledge base, here are a few questions that you can ask:
1. Why people cook at home? Make classification
2. Make classification about what frustrates people about cooking?
3. Brainstorm marketing campaign ideas for an air fryer that would appeal people that cook at home
4. Which kitchen appliences people use most often?
5. What people like about cooking at home?

In [13]:
ask_ai()

What do you want to ask? what is the summary


Response: <b>
Lang chain is a new library that connects AI models to outside sources, allowing users to chain together commands and access personal information. This tutorial series will run through the documentation provided and provide real world code samples and applications. The applications of Lang chain are almost endless and the instructor will emphasize real world applications to help users make an impact in their business or personal life.</b>

What do you want to ask? a good title for the document


Response: <b>
"Getting Started with Lane Chain: A Tutorial Series"</b>

What do you want to ask? what is special about langchain


Response: <b>
Langchain is a library that allows users to connect their AI models to outside sources, allowing them to chain together commands and access information that would otherwise be inaccessible. It is a more recent library that has not been used by many AI models yet, and it allows users to create agents that can do tasks on their behalf, create chatbots, generate questions, answer questions, summarize information, and more. Langchain is special because it allows users to leverage their AI models to access information that would otherwise be inaccessible.</b>

KeyboardInterrupt: ignored